In [ ]:
import os, sys
path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir)))
print(path2add)

if (not (path2add in sys.path)) :
    sys.path.append(path2add)

import BeamTestHelpers as helper
from pathlib import Path
from natsort import natsorted

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms
from tqdm import tqdm
import os
import struct
from natsort import natsorted

In [ ]:
board_titles = {0:'ET2p02_PT_IH14 HV200 OS:20', 1:'ET2p02_PT_IH15 HV200 OS:20', 2:'ET2p02_PT_IH8 HV200 OS:20', 3:'ET2p02_PT_IH6 HV200 OS:20'}
board_titles = {0:'ET2p01_PT_LF6 HV180 OS:20', 1:'ET2p01_PT_LF3 HV180 OS:20', 2:'ET2p01_PT_LF9 HV130 OS:20'}
board_titles = {0:'ET2p02_PT_IH14 HV145 OS:20', 1:'ET2p02_PT_IH15 HV180 OS:20', 2:'ET2p02_PT_IH6 HV180 OS:20', 3:'ET2p02_PT_IH8 HV180 OS:20'}

In [ ]:
path_to_files = "/home/daq/ETROC2_KCU105/ETROC-Data-CE/CE_cosmics_IH_OS20_2/"
path_to_files = "/home/daq/ETROC2_KCU105/ETROC-Data-CE/CE_cosmics_LF_OS20_2/"
path_to_files = '/home/daq/ETROC2_KCU105/ETROC-Data-CE/CC_HPK_test_cosmics_OS20_m20_2'
path_to_files = '/home/daq/ETROC2_KCU105/ETROC-Data-CE/CC_HPKFINAL_cosmics_OS20_RT_0x1_2/'
files = natsorted(Path(path_to_files).glob('file*dat'))
print(files)
df = helper.get_last_complete_event(directory=path_to_files, lines_to_read=-1, hits_per_board={0: 1, 1: 1, 2:1, 3:1})

In [ ]:
df

In [ ]:
# def plot_3d_event_display(df, orientation='left-right', board_order=None, board_titles=None):
#     """
#     Generates a visualization of hits on pixel boards,
#     arranged with a 3D perspective.

#     Args:
#         df (pd.DataFrame): DataFrame containing event data with 'board', 'row', and 'col' columns.
#         orientation (str): 'left-right' for horizontal layout, 'top-down' for vertical.
#         board_order (list): The order in which to display the boards. If None, uses all unique boards from df.
#         board_titles (dict): A dictionary mapping board IDs to custom title strings.
#     """
#     if board_order is None:
#         # If no order is specified, use all unique boards from the dataframe, sorted.
#         board_order = sorted(df['board'].unique())

#     n_boards = len(board_order)
#     if n_boards == 0:
#         print("No boards to display.")
#         return

#     # --- Configuration for the 3D effect ---
#     if orientation == 'left-right':
#         figsize = (6 * n_boards, 8)
#         board_dx = 26
#         board_dy = 0
#         # Skew the horizontal lines downwards for a better 3D effect, keep vertical lines straight.
#         transform = mtransforms.Affine2D().skew_deg(0, -15)
#     elif orientation == 'top-down':
#         figsize = (12, 7 * n_boards)
#         board_dx = 0
#         board_dy = 22
#         # Original transform that works well for the vertical layout
#         transform = mtransforms.Affine2D().skew_deg(-30, 0)
#     else:
#         raise ValueError("orientation must be 'left-right' or 'top-down'")

#     fig, ax = plt.subplots(figsize=figsize, dpi=100)

#     # Colors
#     hit_color = '#3498db'
#     nohit_color = '#ecf0f1'
#     outline_color = '#2c3e50'
#     label_color = '#34495e'
#     line_color = '#1A5276' # Darker blue for the trajectory line

#     # --- Logic for drawing a single-hit trajectory line ---
#     hit_counts_per_board = df['board'].value_counts()
#     is_single_hit_event = all(hit_counts_per_board.get(b, 0) == 1 for b in board_order)
#     hit_coords = {}
#     board_offsets = {}

#     # Loop through the specified board order
#     for i, board_id in enumerate(board_order):
#         # Calculate offsets, compensating for skew
#         offset_y = i * board_dy
#         offset_x = i * board_dx
#         if orientation == 'top-down':
#              # This counteracts the lateral shift caused by skewing 'x' based on 'y'
#             offset_x -= offset_y * np.tan(np.deg2rad(-30))
#         elif orientation == 'left-right':
#             # This counteracts the vertical shift caused by skewing 'y' based on 'x'
#             offset_y -= offset_x * np.tan(np.deg2rad(-15))

#         board_offsets[board_id] = (offset_x, offset_y)

#         # Filter the DataFrame for hits on the current board
#         board_hits = df[df['board'] == board_id]
#         hit_pixels = set(zip(board_hits['row'], board_hits['col']))

#         # If it's a single-hit event, store the center of the hit pixel
#         if is_single_hit_event and not board_hits.empty:
#             row = board_hits['row'].iloc[0]
#             col = board_hits['col'].iloc[0]
#             draw_col = 15 - col
#             center_x = draw_col + offset_x + 0.5
#             center_y = row + offset_y + 0.5
#             hit_coords[board_id] = (center_x, center_y)

#         # Draw the 16x16 grid of pixels for the board
#         for row in range(16):
#             for col in range(16):
#                 is_hit = (row, col) in hit_pixels
#                 color = hit_color if is_hit else nohit_color

#                 # Invert column to place (0,0) at the bottom-right
#                 draw_col = 15 - col

#                 pixel = mpatches.Rectangle(
#                     (draw_col + offset_x, row + offset_y), 1, 1,
#                     facecolor=color, edgecolor=outline_color, linewidth=0.4
#                 )
#                 pixel.set_transform(transform + ax.transData)
#                 ax.add_patch(pixel)

#         # Add a thicker outline for the entire board
#         outline = mpatches.Rectangle(
#             (offset_x, offset_y), 16, 16,
#             facecolor='none', edgecolor=outline_color, linewidth=1.5
#         )
#         outline.set_transform(transform + ax.transData)
#         ax.add_patch(outline)

#         # --- Add Axis Labels and Ticks ---
#         # Add Column labels and ticks (without tick lines)
#         for col in range(16):
#             draw_col = 15 - col
#             label_pos = transform.transform_point((draw_col + offset_x + 0.5, offset_y - 0.8)) # Moved closer
#             ax.text(label_pos[0], label_pos[1], str(col), ha='center', va='top', fontsize=7, color=label_color)

#         # Add Row labels and ticks (without tick lines)
#         for row in range(16):
#             label_pos = transform.transform_point((offset_x - 0.8, row + offset_y + 0.5)) # Moved closer
#             ax.text(label_pos[0], label_pos[1], str(row), ha='right', va='center', fontsize=7, color=label_color)

#         # Add Axis titles ('Row' and 'Column'), aligned with the skewed axes
#         col_angle = np.rad2deg(np.arctan2(transform.get_matrix()[1, 0], transform.get_matrix()[0, 0]))
#         row_angle = np.rad2deg(np.arctan2(transform.get_matrix()[1, 1], transform.get_matrix()[0, 1]))

#         col_title_pos = transform.transform_point((offset_x + 8, offset_y - 2.5))
#         ax.text(col_title_pos[0], col_title_pos[1], 'Column', ha='center', va='center', fontsize=10, color=label_color, rotation=col_angle)

#         row_title_pos = transform.transform_point((offset_x - 2.5, offset_y + 8))
#         ax.text(row_title_pos[0], row_title_pos[1], 'Row', ha='center', va='center', fontsize=10, color=label_color, rotation=row_angle)

#         # Add board labels, using custom titles if provided
#         title = f'Board {board_id}'  # Default title
#         if board_titles and board_id in board_titles:
#             title = board_titles[board_id]

#         if orientation == 'left-right':
#             board_title_pos = transform.transform_point((offset_x + 8, offset_y + 17))
#             ax.text(board_title_pos[0], board_title_pos[1], title, ha='center', va='bottom', fontsize=14, zorder=15, rotation=col_angle)
#         elif orientation == 'top-down':
#             board_title_pos = transform.transform_point((offset_x + 18, offset_y + 8))
#             ax.text(board_title_pos[0], board_title_pos[1], title, ha='left', va='center', fontsize=14, zorder=15, rotation=row_angle + 180)


#     # --- Draw the trajectory line if applicable ---
#     if is_single_hit_event and n_boards >= 2:

#         # The order of iteration depends on the visual layout to get the direction right
#         if orientation == 'top-down':
#             loop_range = range(n_boards - 1, 0, -1)
#             get_board_ids = lambda i: (board_order[i], board_order[i-1])
#         else:
#             loop_range = range(n_boards - 1)
#             get_board_ids = lambda i: (board_order[i], board_order[i+1])

#         for i in loop_range:
#             board1_id, board2_id = get_board_ids(i)

#             if board1_id in hit_coords and board2_id in hit_coords:
#                 p1_raw = np.array(hit_coords[board1_id])
#                 p2_raw = np.array(hit_coords[board2_id])

#                 vector_raw = p2_raw - p1_raw
#                 offset_x1, offset_y1 = board_offsets[board1_id]

#                 if orientation == 'left-right':
#                     exit_plane = offset_x1 + 16
#                     t_crossover = (exit_plane - p1_raw[0]) / vector_raw[0] if vector_raw[0] != 0 else 0.0
#                 else: # top-down
#                     exit_plane = offset_y1
#                     t_crossover = (exit_plane - p1_raw[1]) / vector_raw[1] if vector_raw[1] != 0 else 0.0

#                 t_crossover = np.clip(t_crossover, 0.0, 1.0)
#                 p_crossover_raw = p1_raw + t_crossover * vector_raw

#                 p1_transformed = transform.transform_point(p1_raw)
#                 p_crossover_transformed = transform.transform_point(p_crossover_raw)
#                 p2_transformed = transform.transform_point(p2_raw)

#                 ax.plot([p1_transformed[0], p_crossover_transformed[0]],
#                         [p1_transformed[1], p_crossover_transformed[1]],
#                         color=line_color, linestyle=(0, (8, 4)), linewidth=1.5, zorder=10)

#                 ax.plot([p_crossover_transformed[0], p2_transformed[0]],
#                         [p_crossover_transformed[1], p2_transformed[1]],
#                         color=line_color, linestyle='-', linewidth=2.0, zorder=11, solid_capstyle='round')

#                 # --- Add arrow using annotate ---
#                 solid_vec_transformed = p2_transformed - p_crossover_transformed
#                 if np.linalg.norm(solid_vec_transformed) > 1.0:
#                     arrow_midpoint = p_crossover_transformed + 0.5 * solid_vec_transformed
#                     ax.annotate('', xy=arrow_midpoint, xytext=(arrow_midpoint - 0.1 * solid_vec_transformed),
#                                 arrowprops=dict(arrowstyle="->,head_length=.3,head_width=.2", color=line_color, lw=1.5, shrinkA=0, shrinkB=0),
#                                 zorder=12)

#     # --- Final Plot Adjustments ---
#     ax.autoscale_view()
#     ax.set_aspect('equal', 'box')
#     plt.axis('off')
#     fig.suptitle('3D Event Display', fontsize=18, y=0.98)
#     plt.tight_layout()
#     plt.show()


In [ ]:
def plot_3d_event_display(df, orientation='left-right', board_order=None, board_titles=None, search_region=(6, 6)):
    """
    Generates a visualization of hits on pixel boards,
    arranged with a 3D perspective.
    Draws a reconstructed track if a seed hit is found.

    Args:
        df (pd.DataFrame): DataFrame containing event data with 'board', 'row', and 'col' columns.
        orientation (str): 'left-right' for horizontal layout, 'top-down' for vertical.
        board_order (list): The order in which to display the boards. If None, uses all unique boards from df.
        board_titles (dict): A dictionary mapping board IDs to custom title strings.
        search_region (tuple): A (rows, cols) tuple defining the search window for track reconstruction.
    """
    if board_order is None:
        board_order = sorted(df['board'].unique())

    n_boards = len(board_order)
    if n_boards == 0:
        print("No boards to display.")
        return

    # --- Configuration for the 3D effect ---
    if orientation == 'left-right':
        figsize = (6 * n_boards, 8)
        board_dx = 26
        board_dy = 0
        transform = mtransforms.Affine2D().skew_deg(0, -15)
    elif orientation == 'top-down':
        figsize = (12, 7 * n_boards)
        board_dx = 0
        board_dy = 22
        transform = mtransforms.Affine2D().skew_deg(-30, 0)
    else:
        raise ValueError("orientation must be 'left-right' or 'top-down'")

    fig, ax = plt.subplots(figsize=figsize, dpi=100)

    # Colors
    hit_color = '#3498db'
    nohit_color = '#ecf0f1'
    outline_color = '#2c3e50'
    label_color = '#34495e'
    line_color = '#1A5276'

    # --- Track Reconstruction Logic ---
    hit_counts_per_board = df['board'].value_counts()
    track_hits = {}  # Stores {board_id: (row, col)} for the reconstructed track
    seed_board_id = None
    search_rows, search_cols = search_region

    for board_id in board_order:
        if hit_counts_per_board.get(board_id, 0) == 1:
            seed_board_id = board_id
            break

    if seed_board_id is not None:
        seed_hit = df[df['board'] == seed_board_id].iloc[0]
        track_hits[seed_board_id] = (seed_hit['row'], seed_hit['col'])
        seed_index = board_order.index(seed_board_id)

        # Propagate forward
        last_known_hit = track_hits[seed_board_id]
        for i in range(seed_index + 1, len(board_order)):
            current_board_id = board_order[i]
            candidate_hits = df[df['board'] == current_board_id]
            nearby_hits = candidate_hits[
                (candidate_hits['row'].between(last_known_hit[0] - search_rows, last_known_hit[0] + search_rows)) &
                (candidate_hits['col'].between(last_known_hit[1] - search_cols, last_known_hit[1] + search_cols))
            ]
            if not nearby_hits.empty:
                distances = np.sqrt((nearby_hits['row'] - last_known_hit[0])**2 + (nearby_hits['col'] - last_known_hit[1])**2)
                closest_hit = nearby_hits.loc[distances.idxmin()]
                track_hits[current_board_id] = (closest_hit['row'], closest_hit['col'])
                last_known_hit = track_hits[current_board_id]
            else:
                break

        # Propagate backward
        last_known_hit = track_hits[seed_board_id]
        for i in range(seed_index - 1, -1, -1):
            current_board_id = board_order[i]
            candidate_hits = df[df['board'] == current_board_id]
            nearby_hits = candidate_hits[
                (candidate_hits['row'].between(last_known_hit[0] - search_rows, last_known_hit[0] + search_rows)) &
                (candidate_hits['col'].between(last_known_hit[1] - search_cols, last_known_hit[1] + search_cols))
            ]
            if not nearby_hits.empty:
                distances = np.sqrt((nearby_hits['row'] - last_known_hit[0])**2 + (nearby_hits['col'] - last_known_hit[1])**2)
                closest_hit = nearby_hits.loc[distances.idxmin()]
                track_hits[current_board_id] = (closest_hit['row'], closest_hit['col'])
                last_known_hit = track_hits[current_board_id]
            else:
                break

    hit_coords = {} # Stores the final coordinates for track drawing
    board_offsets = {}

    # --- Drawing Loop ---
    for i, board_id in enumerate(board_order):
        offset_y = i * board_dy
        offset_x = i * board_dx

        if orientation == 'top-down':
            offset_x -= offset_y * np.tan(np.deg2rad(-30))
        elif orientation == 'left-right':
            offset_y -= offset_x * np.tan(np.deg2rad(-15))

        board_offsets[board_id] = (offset_x, offset_y)

        board_hits = df[df['board'] == board_id]
        hit_pixels = set(zip(board_hits['row'], board_hits['col']))

        # Populate hit_coords for the reconstructed track points
        if board_id in track_hits:
            row, col = track_hits[board_id]
            draw_col = 15 - col
            center_x = draw_col + offset_x + 0.5
            center_y = row + offset_y + 0.5
            hit_coords[board_id] = (center_x, center_y)

        for row in range(16):
            for col in range(16):
                is_hit = (row, col) in hit_pixels
                color = hit_color if is_hit else nohit_color
                draw_col = 15 - col
                pixel = mpatches.Rectangle(
                    (draw_col + offset_x, row + offset_y), 1, 1,
                    facecolor=color, edgecolor=outline_color, linewidth=0.4
                )
                pixel.set_transform(transform + ax.transData)
                ax.add_patch(pixel)

        outline = mpatches.Rectangle(
            (offset_x, offset_y), 16, 16, facecolor='none', edgecolor=outline_color, linewidth=1.5
        )
        outline.set_transform(transform + ax.transData)
        ax.add_patch(outline)

        col_angle = np.rad2deg(np.arctan2(transform.get_matrix()[1, 0], transform.get_matrix()[0, 0]))
        row_angle = np.rad2deg(np.arctan2(transform.get_matrix()[1, 1], transform.get_matrix()[0, 1]))

        for col in range(16):
            draw_col = 15 - col
            label_pos = transform.transform_point((draw_col + offset_x + 0.5, offset_y - 0.8))
            ax.text(label_pos[0], label_pos[1], str(col), ha='center', va='top', fontsize=7, color=label_color)

        for row in range(16):
            label_pos = transform.transform_point((offset_x - 0.8, row + offset_y + 0.5))
            ax.text(label_pos[0], label_pos[1], str(row), ha='right', va='center', fontsize=7, color=label_color)

        col_title_pos = transform.transform_point((offset_x + 8, offset_y - 2.5))
        ax.text(col_title_pos[0], col_title_pos[1], 'Column', ha='center', va='center', fontsize=10, color=label_color, rotation=col_angle)

        row_title_pos = transform.transform_point((offset_x - 2.5, offset_y + 8))
        ax.text(row_title_pos[0], row_title_pos[1], 'Row', ha='center', va='center', fontsize=10, color=label_color, rotation=row_angle)

        title = f'Board {board_id}'
        if board_titles and board_id in board_titles:
            title = board_titles[board_id]

        if orientation == 'left-right':
            board_title_pos = transform.transform_point((offset_x + 8, offset_y + 17))
            ax.text(board_title_pos[0], board_title_pos[1], title, ha='center', va='bottom', fontsize=14, zorder=15, rotation=col_angle)
        elif orientation == 'top-down':
            board_title_pos = transform.transform_point((offset_x + 18, offset_y + 8))
            ax.text(board_title_pos[0], board_title_pos[1], title, ha='left', va='center', fontsize=14, zorder=15, rotation=row_angle + 180)


    # --- Draw the trajectory line ---
    if len(track_hits) >= 2:
        if orientation == 'top-down':
            loop_range = range(n_boards - 1, 0, -1)
            get_board_ids = lambda i: (board_order[i], board_order[i-1])
        else:
            loop_range = range(n_boards - 1)
            get_board_ids = lambda i: (board_order[i], board_order[i+1])

        for i in loop_range:
            board1_id, board2_id = get_board_ids(i)
            if board1_id in hit_coords and board2_id in hit_coords:
                p1_raw = np.array(hit_coords[board1_id])
                p2_raw = np.array(hit_coords[board2_id])
                vector_raw = p2_raw - p1_raw
                offset_x1, offset_y1 = board_offsets[board1_id]

                if orientation == 'left-right':
                    exit_plane = offset_x1 + 16
                    t_crossover = (exit_plane - p1_raw[0]) / vector_raw[0] if vector_raw[0] != 0 else 0.0
                else:
                    exit_plane = offset_y1
                    t_crossover = (exit_plane - p1_raw[1]) / vector_raw[1] if vector_raw[1] != 0 else 0.0

                t_crossover = np.clip(t_crossover, 0.0, 1.0)
                p_crossover_raw = p1_raw + t_crossover * vector_raw

                p1_transformed = transform.transform_point(p1_raw)
                p_crossover_transformed = transform.transform_point(p_crossover_raw)
                p2_transformed = transform.transform_point(p2_raw)

                ax.plot([p1_transformed[0], p_crossover_transformed[0]],
                        [p1_transformed[1], p_crossover_transformed[1]],
                        color=line_color, linestyle=(0, (8, 4)), linewidth=1.5, zorder=10)

                ax.plot([p_crossover_transformed[0], p2_transformed[0]],
                        [p_crossover_transformed[1], p2_transformed[1]],
                        color=line_color, linestyle='-', linewidth=2.0, zorder=11, solid_capstyle='round')

                solid_vec_transformed = p2_transformed - p_crossover_transformed
                if np.linalg.norm(solid_vec_transformed) > 1.0:
                    arrow_midpoint = p_crossover_transformed + 0.5 * solid_vec_transformed
                    ax.annotate('', xy=arrow_midpoint, xytext=(arrow_midpoint - 0.1 * solid_vec_transformed),
                                arrowprops=dict(arrowstyle="->,head_length=.3,head_width=.2", color=line_color, lw=1.5, shrinkA=0, shrinkB=0),
                                zorder=12)

    ax.autoscale_view()
    ax.set_aspect('equal', 'box')
    plt.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
if not df.empty:
    print(f"Found a complete event with {len(df)} total hits. Plotting...")
    plot_3d_event_display(df, orientation="left-right",board_order=None, board_titles=board_titles)
else:
    print("No complete event was found that met the criteria.")

In [ ]:
if not df.empty:
    print(f"Found a complete event with {len(df)} total hits. Plotting...")
    plot_3d_event_display(df, orientation="top-down",board_order=None, board_titles=board_titles)
else:
    print("No complete event was found that met the criteria.")